## Explore different `rembg` models

To potentially achieve more accurate background removal, especially for specific types of images, you can experiment with different models provided by the `rembg` library. The default model is `u2net`, which is a general-purpose model.

Here are some commonly used models and their potential use cases:

*   **`u2net` (default)**: A general-purpose model that works well for a wide variety of images.
*   **`u2netp`**: A smaller version of `u2net`, which might be faster but potentially slightly less accurate.
*   **`u2net_human_seg`**: Specifically trained for segmenting humans. This can be very accurate for photos of people.
*   **`isnet-general-use`**: A newer general-purpose model that might perform better on certain images compared to `u2net`.
*   **`isnet-anime`**: Trained specifically for anime-style images.
*   **`sam` (Segment Anything Model)**: A very powerful, large model that can perform accurate segmentation. However, it typically requires more resources and might involve a different workflow or additional dependencies.

You can see the full list of available models by running the following code in a cell:

In the next step, I will show you how to modify the code to select a different model for background removal.

This code does the following:

1.  **Installs Gradio**: The first cell installs the necessary library.
2.  **Imports Libraries**: Imports `gradio`, `remove` and `new_session` from `rembg`, and `PIL` for image handling.
3.  **`remove_background_gradio` Function**: This function takes a PIL Image and a model name as input, uses `rembg` to remove the background, and returns the processed PIL Image.
4.  **Model Options**: It dynamically gets the available models from `rembg` to populate the dropdown.
5.  **Gradio Interface**: It defines the Gradio interface with:
    *   `fn`: The function to run (`remove_background_gradio`).
    *   `inputs`: An image upload component and a dropdown for model selection.
    *   `outputs`: An image display component for the result.
    *   `title` and `description`: Text to display on the web interface.
6.  **Launch**: `iface.launch()` starts the Gradio web server. Gradio will provide a public URL that you can click to access the interface in your browser.

You can run these cells in Colab, and a link will appear in the output that you can click to use the background remover application in your web browser.

In [ ]:
# While directly listing all models programmatically might be complex without inspecting rembg internals
# or specific documentation functions, the common models used with rembg include:
# 'u2net' (default)
# 'u2netp' (a smaller version of u2net)
# 'u2net_human_seg' (specifically trained for human segmentation)
# 'isnet-general-use' (a newer general-purpose model)
# 'isnet-anime' (specifically trained for anime style images)
# 'sam' (Segment Anything Model - requires additional setup and understanding of its usage)

# For the purpose of this exploration subtask, noting the commonly used models is sufficient.
# The next step in the overall task will involve incorporating these into the Gradio interface.

print("Commonly used rembg models:")
print("- 'u2net' (default)")
print("- 'u2netp' (smaller u2net)")
print("- 'u2net_human_seg' (human segmentation)")
print("- 'isnet-general-use' (general purpose)")
print("- 'isnet-anime' (anime images)")
print("- 'sam' (Segment Anything Model - advanced)")

Commonly used rembg models:
- 'u2net' (default)
- 'u2netp' (smaller u2net)
- 'u2net_human_seg' (human segmentation)
- 'isnet-general-use' (general purpose)
- 'isnet-anime' (anime images)
- 'sam' (Segment Anything Model - advanced)


# Task
Add manual background removal using mouse cursor on the processed image with an edit button option at the last in the Gradio UI.

## Modify gradio interface for annotation

### Subtask:
Update the `gr.Image` component in the Gradio interface to enable an annotation mode (e.g., drawing bounding boxes).


**Reasoning**:
I need to modify the `gr.Image` input component in the Gradio interface definition to make it interactive and enable drawing bounding boxes. I will add the `interactive=True` and `tool='boxes'` parameters to the input `gr.Image` component.



In [68]:
import gradio as gr
from rembg import remove, new_session
from PIL import Image, ImageFilter
import io
import numpy as np

# Available models for Gradio dropdown (already defined)
try:
    model_options = list(new_session.AVAILABLE_MODELS.keys())
except AttributeError:
    model_options = ['u2net', 'u2netp', 'u2net_human_seg', 'isnet-general-use', 'isnet-anime']


# Function to remove background using rembg and optionally sharpen
def remove_background_gradio(input_image, model_name='u2net', sharpen_output=False):
    """
    Removes background from an image using the specified rembg model with alpha matting
    and optionally applies a sharpening filter.
    """
    if input_image is None:
        return None

    try:
        session = new_session(model_name)
        img_byte_arr = io.BytesIO()
        input_image.save(img_byte_arr, format='PNG') # Save as PNG to preserve transparency
        img_byte_arr = img_byte_arr.getvalue()

        output_bytes = remove(
            img_byte_arr,
            session=session,
            alpha_matting=True,
            alpha_matting_foreground_threshold=240,
            alpha_matting_background_threshold=10,
            alpha_matting_erode_size=10
        )

        processed_image = Image.open(io.BytesIO(output_bytes)).convert('RGBA')

        # --- Conditional Sharpening Step ---
        if sharpen_output:
            # Split into RGB and Alpha channels
            rgb_image = processed_image.convert('RGB')
            alpha_channel = processed_image.getchannel('A')

            # Apply sharpening filter to the RGB image
            sharpened_rgb_image = rgb_image.filter(ImageFilter.SHARPEN)

            # Recombine the sharpened RGB image with the original alpha channel
            processed_image = Image.merge('RGBA', (sharpened_rgb_image.split() + (alpha_channel,)))
            # --- End Conditional Sharpening Step ---


        # Return the processed RGBA image (sharpened or not)
        return processed_image

    except Exception as e:
        print(f"Error during background removal or sharpening: {e}")
        return None


# Function to prepare image for download
def prepare_image_for_download(image: Image.Image):
    """
    Saves a PIL Image to a BytesIO object and returns its byte content for download.
    """
    if image is None:
        return None
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG') # Save as PNG to preserve transparency
    img_byte_arr.seek(0) # Seek to the beginning of the BytesIO object
    return img_byte_arr.getvalue() # Return the byte content


# Create the Gradio Blocks interface
with gr.Blocks(title="AI Background Remover with Sharpening and Download") as iface:
    gr.Markdown("# AI Background Remover with Sharpening and Download")
    gr.Markdown("Upload an image, automatically remove the background, optionally sharpen, and download the result.")

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Upload Image")
            model_dropdown = gr.Dropdown(choices=model_options, value='u2net', label="Select AI Model")
            sharpen_checkbox = gr.Checkbox(label="Apply Sharpening", value=False)
            remove_button = gr.Button("Remove Background")

        with gr.Column():
            processed_image = gr.Image(type="pil", label="Processed Image with Background Removed", interactive=False)
            download_button = gr.Button("Download Processed Image")
            download_file = gr.File(label="Download File")


    # Define the workflow
    remove_button.click(
        fn=remove_background_gradio,
        inputs=[input_image, model_dropdown, sharpen_checkbox],
        outputs=[processed_image]
    )

    # Link the download button to the function that prepares the image for download
    download_button.click(
        fn=prepare_image_for_download,
        inputs=[processed_image],
        outputs=[download_file]
    )


# Launch the Gradio app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40ab12b0f6f737e72c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
